In [1]:
# important packages
import json
import requests
import pandas as pd
import numpy as np

# Random Stuff to Test How Pulling From Asset API Works

In [18]:
# url
api_endpoint = 'https://wax.api.atomicassets.io/atomicassets' 
# api key
api_request = '/v1/assets?collection_name=mlb.topps&limit=100' 

url = api_endpoint + api_request

In [19]:
first_pull = requests.get(url).json()

In [20]:
first_pull['data'][19]

{'contract': 'atomicassets',
 'asset_id': '1099554126259',
 'owner': 'n4nqw.wam',
 'is_transferable': True,
 'is_burnable': True,
 'collection': {'collection_name': 'mlb.topps',
  'name': '2021 Topps Series 1 Baseball',
  'img': 'QmStHeaGaVMQDncrs6KVg5Jrc9Q8chNj8u6QvCN96js2Qb',
  'author': 'mlb.topps',
  'allow_notify': True,
  'authorized_accounts': ['mlb.topps',
   '4je.m.wam',
   'config.nft',
   'unbox.nft',
   'sales.nft',
   'craft.nft',
   'premint.nft',
   'dk2ausetup11'],
  'notify_accounts': [],
  'market_fee': 0.06,
  'created_at_block': '113636159',
  'created_at_time': '1618257661500'},
 'schema': {'schema_name': 'promo',
  'format': [{'name': 'name', 'type': 'string'},
   {'name': 'rarity', 'type': 'string'},
   {'name': 'img', 'type': 'image'},
   {'name': 'variant', 'type': 'string'},
   {'name': 'artist', 'type': 'string'},
   {'name': 'legal', 'type': 'string'}],
  'created_at_block': '114322296',
  'created_at_time': '1618601222500'},
 'template': {'template_id': '10

In [28]:
# url
api_endpoint = 'https://wax.api.atomicassets.io/atomicassets' 
# api key
api_request = '/v1/assets?collection_name=mlb.topps&page=1500000&limit=1' 

url = api_endpoint + api_request

In [29]:
test_pull2 = requests.get(url).json()['data']
test_pull2[0]

{'contract': 'atomicassets',
 'asset_id': '1099525276039',
 'owner': '4hdbq.wam',
 'is_transferable': True,
 'is_burnable': True,
 'collection': {'collection_name': 'mlb.topps',
  'name': '2021 Topps Series 1 Baseball',
  'img': 'QmStHeaGaVMQDncrs6KVg5Jrc9Q8chNj8u6QvCN96js2Qb',
  'author': 'mlb.topps',
  'allow_notify': True,
  'authorized_accounts': ['mlb.topps',
   '4je.m.wam',
   'config.nft',
   'unbox.nft',
   'sales.nft',
   'craft.nft',
   'premint.nft',
   'dk2ausetup11'],
  'notify_accounts': [],
  'market_fee': 0.06,
  'created_at_block': '113636159',
  'created_at_time': '1618257661500'},
 'schema': {'schema_name': 'packs',
  'format': [{'name': 'name', 'type': 'string'},
   {'name': 'img', 'type': 'image'},
   {'name': 'video', 'type': 'image'},
   {'name': 'series', 'type': 'string'},
   {'name': 'contains', 'type': 'string'},
   {'name': 'unpack_url', 'type': 'string'},
   {'name': 'description', 'type': 'string'},
   {'name': 'legal', 'type': 'string'}],
  'created_at_bl

# Python Script

In [ ]:
# make an empty pandas df
assets_database = pd.DataFrame()
sales_database = pd.DataFrame()

# each page contains one NFT asset - each request will obtain one page
page_counter = 1

# api endpoint for ASSETS
asset_endpoint = 'https://wax.api.atomicassets.io/atomicassets'

# api endpoint for SALES
sales_endpoint = 'https://wax.api.atomicassets.io/atomicmarket'

# PERFORM UNIT TESTING 

while True:
    # ADD A SLEEP FUNCTION FOR AN HOUR 
    # CONSIDER rate limits - upping the limit to a thousands
    
    # api key for ASSETS 
    asset_key = '/v1/assets?collection_name=mlb.topps&page={page_num}&limit=1000'.format(page_num = page_counter)
    
    # form the asset api to pass into the request
    asset_api = asset_endpoint + asset_key
    
    # pull an asset from the API  
    asset_pull = requests.get(asset_api).json()['data']
    
    # exit the loop if there are no more assets to pull
    if len(asset_pull) < 1000: 
        # add the rest of the
        break
        
    # iterate through the array of assets - add each asset to a pandas csv 
    
    # itearte through each of the 1000 assets in the pull and get their sales via asset id 
    # put this in a loop - ADD ANOTHER SLEEP FUNCION IN THE MARKET LOOP 
    # 
    
    # obtain the asset from the api pull request
    asset = asset_pull[0]
    
    # LINE OF CODE TO SOMEHOW STORE THE ASSET (CSV, ARRAY, ETC.)
    asset_database = ...
    
    # obtain the id of the current asset 
    asset_id = asset['asset_id']
    
    # api key for SALES of a given asset
    sales_key = ... # see cells below 
    
    # form the sales api to pass into the request
    sales_api = sales_endpoint + sales_key
    
    # pull the sales of that ASSET from the API
    sales_pull = requests.get(sales_api).json['data']
    
    # 
    # ADD TO PANDAS DATABASE
    
    # increment the page counter to pull a different asset on the next iteration
    page_counter += 1

# Testing Sales

In [148]:
# api endpoint
sales_endpoint = 'https://wax.api.atomicassets.io/atomicmarket'

In [172]:
# add a state for sold, listed, etc.. ask the team about this 
# STATE parameter - NOT NEEDED
sales_key = 'https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666'
url = sales_endpoint + sales_key
url

'https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666'

In [174]:
x = requests.get('https://wax.api.atomicassets.io/atomicmarket/v1/sales?asset_id=1099554263666&state=3').json()['data']
x

[]